In [1]:
# https://towardsdatascience.com/hands-on-generative-adversarial-networks-gan-for-signal-processing-with-python-ff5b8d78bd28

In [2]:
from numpy import hstack
from numpy import zeros
from numpy import ones
from numpy.random import rand
from numpy.random import randn
import numpy as np
import pandas as pd
from sklearn.utils import resample
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder
from collections import Counter
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Input
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM
from matplotlib import pyplot
import matplotlib.pyplot as plt

In [3]:
# prepare real data
%run ./read_file.ipynb

:: loading settings :: url = jar:file:/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ec2-user/.ivy2/cache
The jars for the packages stored in: /home/ec2-user/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-7f3622b3-4c55-4558-8b28-119e8c5f7c92;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 135ms :: artifacts dl 4ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	-----------------------------

23/04/12 23:11:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/12 23:11:49 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [4]:
data = spark.read.parquet(*["s3a://sapient-bucket-trusted/prod/graph/encoded/real/23Sep3/*"]).cache()

In [5]:
# https://stackoverflow.com/questions/68036975/valueerror-shape-must-be-at-least-rank-3-but-is-rank-2-for-node-biasadd
# config for rank error in lstm
tf.keras.backend.set_image_data_format("channels_last")

In [6]:
# Set Config
embedding_dim = 64
max_length = 6
sequence_length = 6
max_features = 10000
padding_type = 'post'
trunc_type = 'post'
training_portion = 0.8

In [7]:
# number of samples to make
input_len = 1

In [43]:
def prepare_real_samples(ds, lc=300):
    # Calculate the number of malicious and non-malicious records
    malicious_ds = ds.filter(col("malicious") == 1).limit(lc)
    ds_events = malicious_ds.select('event_sequence').rdd.flatMap(lambda x: x).collect()
    tokenizer = tf.keras.preprocessing.text.Tokenizer()
    # Get our training data word index
    tokenizer.fit_on_texts(ds_events)
    word_index = tokenizer.word_index
    vocab_count = len(word_index)
    train_sequences = tokenizer.texts_to_sequences(ds_events)
    train_padded = tf.keras.utils.pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
    train_ds = tf.reshape(train_padded, (-1, len(train_padded), 6, 1, 1))
    train_labels_ds = ones((1, input_len, 6, 1, 1))
    return train_ds, train_labels_ds, word_index

In [9]:
METRICS = [
        keras.metrics.TruePositives(name='tp'),
        keras.metrics.FalsePositives(name='fp'),
        keras.metrics.TrueNegatives(name='tn'),
        keras.metrics.FalseNegatives(name='fn'), 
        keras.metrics.BinaryAccuracy(name='accuracy'),
        keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall'),
        keras.metrics.AUC(name='auc'),
        keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

In [10]:
def rescale_value(val, old_range, new_range):
    """
    this converts a value within a range of values to an integer within another range 
    """
    new_val = tf.cast((val - old_range[0]) / (old_range[1] - old_range[0]) * (new_range[1] - new_range[0]) + new_range[0],tf.int32)
    return new_val

In [11]:
# define the standalone generator model
# https://www.tensorflow.org/tutorials/generative/dcgan#the_generator
def define_generator():
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(input_len, 6)),
        tf.keras.layers.LSTM(64, return_sequences=True),
        tf.keras.layers.Dense(6, activation='sigmoid'),
        # tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1)),
    ])
    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=METRICS)
    return model

In [12]:
# define the standalone discriminator model
def define_discriminator():
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(input_len, 6, 1)), # Flatten the input tensor
        tf.keras.layers.Embedding(100 + 1, 16),
        tf.keras.layers.Reshape((input_len, 6, 16)), # Reshape embedding output to match Dense output
        tf.keras.layers.Flatten(), # Flatten the output of the Embedding layer
        tf.keras.layers.Dense(input_len * 6, activation='tanh'), # Add an additional Dense layer to get desired output shape
        tf.keras.layers.Reshape((input_len, 6, 1)), # Reshape output to desired shape

    ])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=METRICS)
    return model

In [13]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(generator, discriminator):
    # connect them
    model = Sequential()
    model.add(generator)
    model.add(discriminator)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=METRICS)
    return model

In [14]:
def generate_latent_space():
    n = tf.random.uniform(shape=[input_len, 6, 1], minval=1, maxval=100, dtype=tf.int32)
    n_ds = tf.reshape(n, (-1, input_len, 6, 1))
    return n_ds

In [15]:
def generate_fake_samples(generator, latent_dim=3, n=0):
    # generate points in latent space
    n_ds = generate_latent_space()
    # predict outputs
    X = generator.predict(n_ds, verbose=0)
    # print("size of output from generator")
    # print(X.shape)
    # add extra dimension to output tensor
    X = tf.expand_dims(X, axis=-1)
    # print("size of input to discriminator after expansion")
    # print(X.shape)
    # create class labels
    y = zeros((1, input_len, 6, 1, 1))
    # print("size of labels to discriminator")
    # print(y.shape)
    return X, y

In [16]:
metrics = ["accuracy", "loss", "prc", "recall", "auc", "precision"]
def plot_graphs(history, metrics):
    fig, axs = plt.subplots(nrows = 2, ncols=3, figsize=(12, 6), layout="constrained")
    for ax, m in zip(axs.flat, metrics):
        ax.plot(history.history[m])
        ax.plot(history.history['val_'+ m])
        ax.set_title(f'{m}')

In [17]:
# use the generator to generate n fake examples, with class labels
def train(g_model, d_model, gan_model, latent_dim=3, n_epochs=10, n_batch=128, n_eval=20):
    # determine half the size of one batch, for updating the discriminator
    half_batch = int(n_batch / 2)
    # manually enumerate epochs
    for i in range(n_epochs):
        # prepare real samples
        # x_real, y_real = generate_fake_samples(g_model, input_len)
        x_real, y_real, vocab = prepare_real_samples(data, lc=input_len)
        # prepare fake examples using the generator
        x_fake, y_fake = generate_fake_samples(g_model, input_len)
        # update discriminator
        d_model.train_on_batch(x_real, y_real)
        d_model.train_on_batch(x_fake, y_fake)
        # prepare points in latent space as input for the generator
        x_gan = generate_latent_space()
        # create inverted labels for the fake samples
        y_gan = ones((1, input_len, 6, 1))
        # create inverted labels for the fake samples
        # update the generator via the discriminator's error
        gan_model.train_on_batch(x_gan, y_gan)
        # evaluate the model every n_eval epochs
        if (i+1) % n_eval == 0:
            print('Number of epochs = %i'%(i+1))

In [18]:
# Test combination of generator and discriminator
generator = define_generator()
discriminator = define_discriminator()
gan = define_gan(generator, discriminator)

In [19]:
# test gan created data and model

In [20]:
x_gan = generate_latent_space()
# x_gan = tf.reshape(x_gan, (-1, input_len, 6, 1, 1))
# create inverted labels for the fake samples
y_gan = ones(([1,input_len, 6, 1, 1]))

In [21]:
x_gan.shape

TensorShape([1, 1, 6, 1])

In [22]:
y_gan.shape

(1, 1, 6, 1, 1)

In [23]:
discriminator.train_on_batch(x_gan, y_gan)

[9.70156478881836, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 1.0]

In [24]:
# Create real gan

In [25]:
train(generator, discriminator, gan, n_epochs=20)

Number of epochs = 20


In [38]:
# create data based on the trained model
si = generate_latent_space()
sp = gan.predict(si)

1/1 [==============================] - 0s 23ms/step


In [39]:
sp.shape

(1, 1, 6, 1)

In [28]:
sp[0]

<tf.Tensor: shape=(6, 1), dtype=float32, numpy=
array([[ 0.08062109],
       [-0.02939204],
       [ 0.12279744],
       [-0.08814587],
       [-0.01482284],
       [ 0.11645006]], dtype=float32)>

In [29]:
tf.random.categorical( sp, num_samples=1)[-1, 0].numpy()[0]

InvalidArgumentError: {{function_node __wrapped__Multinomial_device_/job:localhost/replica:0/task:0/device:GPU:0}} logits should be a matrix, got shape [1,6,1] [Op:Multinomial]

In [30]:
sp[0][1]

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([-0.02939204], dtype=float32)>

In [31]:
sp.shape

TensorShape([1, 6, 1])

In [32]:
len(sp_ds)

NameError: name 'sp_ds' is not defined

In [33]:
#sp_ds = tf.reshape(sp, (input_len, 6, 1))

In [34]:
np.max(si)

83

In [35]:
np.max(sp_ds)

NameError: name 'sp_ds' is not defined

In [36]:
np.min(sp_ds)

NameError: name 'sp_ds' is not defined

In [37]:
print(sp_ds[20:])

NameError: name 'sp_ds' is not defined

In [41]:
table = tf.lookup.StaticHashTable(
    tf.lookup.KeyValueTensorInitializer(
        keys=vocab, values=tf.constant(list(range(vocab_count)),
                                       dtype=tf.int64)),
    default_value=0)

NameError: name 'vocab' is not defined